In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import psycopg2
import sys
import socket
import datetime

def second_page(sd):
    driver.find_element_by_xpath(u"(//input[@value='詳細資料'])[" + str(sd) + "]").click()

    time.sleep(6)
    now_handle=driver.current_window_handle
    all_handles=driver.window_handles
    driver.switch_to.window(all_handles[1])
    aa=driver.page_source
    driver.close()
    driver.switch_to.window(all_handles[0])
    soup=BeautifulSoup(aa,'html5lib')
    print ("soup of second_page :")
    print (soup)
    print("\n")
    
    return soup

def parser(soup):
    compID=soup.find('input',{'name':'Q1V'})['value']
    compName=soup.find('input',{'name':'compName'})['value'].replace(' ','')
    yyymm=soup.find('input',{'name':'Q2V'})['value']
    mamaID=compID[0:4]
    timestamp=time.strftime("%Y-%m-%d %H:%M:%S")
    print("id=" + compID + "; compName=" + compName + "; yyymm=" + yyymm)

    name=[]
    name.append('非持有_符合避險_未沖銷_總金額')
    name.append('非持有_符合避險_未沖銷_公允價值')
    name.append('非持有_符合避險_未沖銷_未實現')
    name.append('非持有_符合避險_已沖銷_總金額')
    name.append('非持有_符合避險_已沖銷_已實現')
    name.append('非持有_不符合避險_未沖銷_總金額')
    name.append('非持有_不符合避險_未沖銷_公允價值')
    name.append('非持有_不符合避險_未沖銷_未實現')
    name.append('非持有_不符合避險_已沖銷_總金額')
    name.append('非持有_不符合避險_已沖銷_已實現')

    td=[]
    for i in range(0,10):
        td.append(0)

    for tb in soup.findAll('table',{'class':'hasBorder'}):
        a1=tb.findAll('tr',{'class':False})[0].findAll('td',{'class':'tblHead'})[0].text
        #print(first_head)
        if a1=='非持有供交易':
            a2=tb.findAll('tr',{'class':False})[0].findAll('td',{'class':'tblHead'})[1].text
            if a2=='不符避險會計':
                td[5]=tb.findAll('tr',{'class':False})[2].findAll('td',{'style':True})[-4].text.replace(',','').replace(' ','')
                td[6]=tb.findAll('tr',{'class':False})[3].findAll('td',{'style':True})[-4].text.replace(',','').replace(' ','')
                td[7]=tb.findAll('tr',{'class':False})[4].findAll('td',{'style':True})[-4].text.replace(',','').replace(' ','')
                td[8]=tb.findAll('tr',{'class':False})[5].findAll('td',{'style':True})[-4].text.replace(',','').replace(' ','')
                td[9]=tb.findAll('tr',{'class':False})[6].findAll('td',{'style':True})[-4].text.replace(',','').replace(' ','')
            elif a2=='符合避險會計':
                td[0]=tb.findAll('tr',{'class':False})[2].findAll('td',{'style':True})[-4].text.replace(',','').replace(' ','')
                td[1]=tb.findAll('tr',{'class':False})[3].findAll('td',{'style':True})[-4].text.replace(',','').replace(' ','')
                td[2]=tb.findAll('tr',{'class':False})[4].findAll('td',{'style':True})[-4].text.replace(',','').replace(' ','')
                td[3]=tb.findAll('tr',{'class':False})[5].findAll('td',{'style':True})[-4].text.replace(',','').replace(' ','')
                td[4]=tb.findAll('tr',{'class':False})[6].findAll('td',{'style':True})[-4].text.replace(',','').replace(' ','')
    isum=0
    for i in range(0,10):
        isum=isum+ int(td[i])
       
    if isum!=0:
        for i in range(0,10):
            cur.execute("insert into 遠期契約申報資料(id,mama_id,name,yyymm,item,value,insert_dt) values(" + compID + "," + mamaID + ",'" + compName + "'," + yyymm + ",'" + name[i] + "'," + str(td[i]) + ",'" + timestamp + "')")   
            conn.commit()

def isExist(soup):
    compID=soup.find('input',{'name':'Q1V'})['value']
    yyymm=soup.find('input',{'name':'Q2V'})['value']
    mamaID=compID[0:4]
    
   # cur.execute("select count(id) from 遠期契約申報資料 where id='" + mamaID + "' and yyymm='" + yyymm + "';")   
  #  arr=cur.fetchall()
   # conn.commit()
  #  return str(arr[0][0])
    return '0'

def start_level(sp):
    if isExist(sp)=='0': 
        all_lists=0
        for i in sp.findAll('input',{'type':'button','value':'詳細資料'}):
            all_lists=all_lists+1

        isSuccess=False
        iStart=1
        recursive=0
        while(isSuccess==False):
            try:
                for i in range(iStart,all_lists+1):
                    print("to the range of \'start_level try range\'\n")
                    soup=second_page(i)
                    print("start_level while of soup"+soup)
                    print("\n")
               #     parser(soup)

                isSuccess=True
                print('Mission complete!!!')
            except:
                if recursive<3:
                    iStart=i
                    recursive=recursive+1
                    print('stop at ' + str(iStart) + ', and recursive times = ' + str(recursive))
                else:
                    iStart=i+1
                    recursive=0
                
        driver.close()
    else:
        print('Already exists!!!')
        driver.close()

        
def trigger(myID, myYYY, myMM):
 #   if socket.gethostbyname(socket.gethostname())=='172.27.110.104':	#Jack's pc
   #     ipath='D:\\Python36\\phantomjs.exe'
  #  elif socket.gethostbyname(socket.gethostname())=='172.27.111.34': 	#Jay's pc
   #     ipath='C:\\Program Files\\Python35\\Scripts\\phantomjs.exe'
   # elif socket.gethostbyname(socket.gethostname())=='172.27.110.105': 	#Allen's pc
   #     ipath='D:\\Python\\Python35\\phantomjs.exe'
    ipath='D:\\financial_data_web_scrawer\\chromedriver.exe'
		
    global driver
    driver=webdriver.Chrome(ipath,service_args=['--ignore-ssl-errors=true','--ssl-protocol=TLSv1'])

   # driver=webdriver.Firefox()
    driver.get("http://mops.twse.com.tw/mops/web/t15sf")
    print("myID:"+myID)
    print("\n")
    print("year:"+myYYY)
    print("\n")
    print("month:"+myMM)
    print("\n")
    driver.find_element_by_id('co_id').send_keys(myID)
    driver.find_element_by_id('year').send_keys(myYYY)
    driver.find_element_by_id('month').send_keys(int(myMM))
    #Select(driver.find_element_by_id('month')).select_by_visible_text(int(myMM))
    driver.find_element_by_css_selector('div.search > input[type=\"button\"]').click()
    time.sleep(6)
    soup=BeautifulSoup(driver.page_source,'html5lib')
   # print(soup)
    is_no_data=False
    for center in soup.findAll('center'):
        if center.text=='該公司並未申報本項資料':
            is_no_data=True
    
    if is_no_data:
        print('該公司並未申報本項資料')
    else:
        start_level(soup)		
		
def get_all_code(strName):
    host_str=connection_info()
    conn1=psycopg2.connect(host=host_str,port='5432',database='equity',user='quant',password='quant')
    cur1=conn1.cursor()
    cur1.execute("select id from " + strName + " order by id")   
    arr=cur1.fetchall()
    cur1.close()
    conn1.close()
    return arr
	
#add id and date to trrigger 
def iteration(arr, start_from=0, end_to=9999):
    lastmon=datetime.datetime.today()-datetime.timedelta(days=100)
    myYYY=str(int(lastmon.strftime("%Y"))-1911)
    myMM=lastmon.strftime("%m")
    myID=''
    
    for i in range(0,len(arr)):
        myID='1540'
        if int(myID)>=start_from and int(myID)<=end_to:
            trigger(myID, myYYY, myMM)
# connect to local db
def connection_info():
    #ipstr=socket.gethostbyname(socket.gethostname())
    #if ipstr=='172.27.110.104':
    #    conn_info='localhost'
    #else:
    #    conn_info='172.27.110.104'  #this is Jack's pc
    conn_info='localhost'
    return conn_info
lastmon=datetime.datetime.today()-datetime.timedelta(days=100)
print("lastmon"+str(lastmon))
myYYY=str(int(lastmon.strftime("%Y"))-1911)
myMM=lastmon.strftime("%m")
myID=''

#host_str=connection_info()
#conn=psycopg2.connect(host=host_str,port='5432',database='postgres',user='quant',password='quant')
#cur=conn.cursor()

arr=['1']
iteration(arr)  	# for debug purpose



lastmon2019-02-18 12:24:11.070736
myID:1540


year:108


month:02


to the range of 'start_level try range'

soup of second_page :
<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="js/mops.js" type="text/javascript"></script><link href="css/css.css" rel="stylesheet" type="text/css"/></head><body style="background-color:white !important;"><div id="nav02"><div class="t01" id="caption"><br/>   衍生性商品交易資訊</div><div id="zoom"><div id="table01">



	<title>公開資訊觀測站</title>
<!--	<link href="css/css1.css" rel="stylesheet" type="text/css" Media="Screen"/> -->
<!--	<script type="text/javascript" src="js/mops1.js"></script> -->



<!--_co_id_hhc=1540__-->
<!-- new screen -->
<table align="right" class="noBorder">
<tbody><tr><td align="right">
<form action="/server-java/t56ques" method="post">
<input name="step" type="hidden" value="0"/>
<input name="Market" type="hidden" value="sii"/>
<input name="SysName" type="hidden" value="營運概況"/>
<input name="repo

soup of second_page :
<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="js/mops.js" type="text/javascript"></script><link href="css/css.css" rel="stylesheet" type="text/css"/></head><body style="background-color:white !important;"><div id="nav02"><div class="t01" id="caption"><br/>   衍生性商品交易資訊</div><div id="zoom"><div id="table01">



	<title>公開資訊觀測站</title>
<!--	<link href="css/css1.css" rel="stylesheet" type="text/css" Media="Screen"/> -->
<!--	<script type="text/javascript" src="js/mops1.js"></script> -->



<!--_co_id_hhc=1540__-->
<!-- new screen -->
<table align="right" class="noBorder">
<tbody><tr><td align="right">
<form action="/server-java/t56ques" method="post">
<input name="step" type="hidden" value="0"/>
<input name="Market" type="hidden" value="sii"/>
<input name="SysName" type="hidden" value="營運概況"/>
<input name="reportName" type="hidden" value="衍生性商品交易資訊"/>
<input name="colorchg" type="hidden" value=""/>
<input name="compID

soup of second_page :
<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="js/mops.js" type="text/javascript"></script><link href="css/css.css" rel="stylesheet" type="text/css"/></head><body style="background-color:white !important;"><div id="nav02"><div class="t01" id="caption"><br/>   衍生性商品交易資訊</div><div id="zoom"><div id="table01">



	<title>公開資訊觀測站</title>
<!--	<link href="css/css1.css" rel="stylesheet" type="text/css" Media="Screen"/> -->
<!--	<script type="text/javascript" src="js/mops1.js"></script> -->



<!--_co_id_hhc=1540__-->
<!-- new screen -->
<table align="right" class="noBorder">
<tbody><tr><td align="right">
<form action="/server-java/t56ques" method="post">
<input name="step" type="hidden" value="0"/>
<input name="Market" type="hidden" value="sii"/>
<input name="SysName" type="hidden" value="營運概況"/>
<input name="reportName" type="hidden" value="衍生性商品交易資訊"/>
<input name="colorchg" type="hidden" value=""/>
<input name="compID

soup of second_page :
<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="js/mops.js" type="text/javascript"></script><link href="css/css.css" rel="stylesheet" type="text/css"/></head><body style="background-color:white !important;"><div id="nav02"><div class="t01" id="caption"><br/>   衍生性商品交易資訊</div><div id="zoom"><div id="table01">



	<title>公開資訊觀測站</title>
<!--	<link href="css/css1.css" rel="stylesheet" type="text/css" Media="Screen"/> -->
<!--	<script type="text/javascript" src="js/mops1.js"></script> -->



<!--_co_id_hhc=1540__-->
<!-- new screen -->
<table align="right" class="noBorder">
<tbody><tr><td align="right">
<form action="/server-java/t56ques" method="post">
<input name="step" type="hidden" value="0"/>
<input name="Market" type="hidden" value="sii"/>
<input name="SysName" type="hidden" value="營運概況"/>
<input name="reportName" type="hidden" value="衍生性商品交易資訊"/>
<input name="colorchg" type="hidden" value=""/>
<input name="compID

soup of second_page :
<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="js/mops.js" type="text/javascript"></script><link href="css/css.css" rel="stylesheet" type="text/css"/></head><body style="background-color:white !important;"><div id="nav02"><div class="t01" id="caption"><br/>   衍生性商品交易資訊</div><div id="zoom"><div id="table01">



	<title>公開資訊觀測站</title>
<!--	<link href="css/css1.css" rel="stylesheet" type="text/css" Media="Screen"/> -->
<!--	<script type="text/javascript" src="js/mops1.js"></script> -->



<!--_co_id_hhc=1540001__-->
<!-- new screen -->
<table align="right" class="noBorder">
<tbody><tr><td align="right">
<form action="/server-java/t56ques" method="post">
<input name="step" type="hidden" value="0"/>
<input name="Market" type="hidden" value="sii"/>
<input name="SysName" type="hidden" value="營運概況"/>
<input name="reportName" type="hidden" value="衍生性商品交易資訊"/>
<input name="colorchg" type="hidden" value=""/>
<input name="com

...

soup of second_page :
<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="js/mops.js" type="text/javascript"></script><link href="css/css.css" rel="stylesheet" type="text/css"/></head><body style="background-color:white !important;"><div id="nav02"><div class="t01" id="caption"><br/>   衍生性商品交易資訊</div><div id="zoom"><div id="table01">



	<title>公開資訊觀測站</title>
<!--	<link href="css/css1.css" rel="stylesheet" type="text/css" Media="Screen"/> -->
<!--	<script type="text/javascript" src="js/mops1.js"></script> -->



<!--_co_id_hhc=1540002__-->
<!-- new screen -->
<table align="right" class="noBorder">
<tbody><tr><td align="right">
<form action="/server-java/t56ques" method="post">
<input name="step" type="hidden" value="0"/>
<input name="Market" type="hidden" value="sii"/>
<input name="SysName" type="hidden" value="營運概況"/>
<input name="reportName" type="hidden" value="衍生性商品交易資訊"/>
<input name="colorchg" type="hidden" value=""/>
<input name="com

soup of second_page :
<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="js/mops.js" type="text/javascript"></script><link href="css/css.css" rel="stylesheet" type="text/css"/></head><body style="background-color:white !important;"><div id="nav02"><div class="t01" id="caption"><br/>   衍生性商品交易資訊</div><div id="zoom"><div id="table01">



	<title>公開資訊觀測站</title>
<!--	<link href="css/css1.css" rel="stylesheet" type="text/css" Media="Screen"/> -->
<!--	<script type="text/javascript" src="js/mops1.js"></script> -->



<!--_co_id_hhc=1540003__-->
<!-- new screen -->
<table align="right" class="noBorder">
<tbody><tr><td align="right">
<form action="/server-java/t56ques" method="post">
<input name="step" type="hidden" value="0"/>
<input name="Market" type="hidden" value="sii"/>
<input name="SysName" type="hidden" value="營運概況"/>
<input name="reportName" type="hidden" value="衍生性商品交易資訊"/>
<input name="colorchg" type="hidden" value=""/>
<input name="com

soup of second_page :
<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="js/mops.js" type="text/javascript"></script><link href="css/css.css" rel="stylesheet" type="text/css"/></head><body style="background-color:white !important;"><div id="nav02"><div class="t01" id="caption"><br/>   衍生性商品交易資訊</div><div id="zoom"><div id="table01">



	<title>公開資訊觀測站</title>
<!--	<link href="css/css1.css" rel="stylesheet" type="text/css" Media="Screen"/> -->
<!--	<script type="text/javascript" src="js/mops1.js"></script> -->



<!--_co_id_hhc=1540003__-->
<!-- new screen -->
<table align="right" class="noBorder">
<tbody><tr><td align="right">
<form action="/server-java/t56ques" method="post">
<input name="step" type="hidden" value="0"/>
<input name="Market" type="hidden" value="sii"/>
<input name="SysName" type="hidden" value="營運概況"/>
<input name="reportName" type="hidden" value="衍生性商品交易資訊"/>
<input name="colorchg" type="hidden" value=""/>
<input name="com

soup of second_page :
<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="js/mops.js" type="text/javascript"></script><link href="css/css.css" rel="stylesheet" type="text/css"/></head><body style="background-color:white !important;"><div id="nav02"><div class="t01" id="caption"><br/>   衍生性商品交易資訊</div><div id="zoom"><div id="table01">



	<title>公開資訊觀測站</title>
<!--	<link href="css/css1.css" rel="stylesheet" type="text/css" Media="Screen"/> -->
<!--	<script type="text/javascript" src="js/mops1.js"></script> -->



<!--_co_id_hhc=1540003__-->
<!-- new screen -->
<table align="right" class="noBorder">
<tbody><tr><td align="right">
<form action="/server-java/t56ques" method="post">
<input name="step" type="hidden" value="0"/>
<input name="Market" type="hidden" value="sii"/>
<input name="SysName" type="hidden" value="營運概況"/>
<input name="reportName" type="hidden" value="衍生性商品交易資訊"/>
<input name="colorchg" type="hidden" value=""/>
<input name="com

soup of second_page :
<html><head><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="js/mops.js" type="text/javascript"></script><link href="css/css.css" rel="stylesheet" type="text/css"/></head><body style="background-color:white !important;"><div id="nav02"><div class="t01" id="caption"><br/>   衍生性商品交易資訊</div><div id="zoom"><div id="table01">



	<title>公開資訊觀測站</title>
<!--	<link href="css/css1.css" rel="stylesheet" type="text/css" Media="Screen"/> -->
<!--	<script type="text/javascript" src="js/mops1.js"></script> -->



<!--_co_id_hhc=1540003__-->
<!-- new screen -->
<table align="right" class="noBorder">
<tbody><tr><td align="right">
<form action="/server-java/t56ques" method="post">
<input name="step" type="hidden" value="0"/>
<input name="Market" type="hidden" value="sii"/>
<input name="SysName" type="hidden" value="營運概況"/>
<input name="reportName" type="hidden" value="衍生性商品交易資訊"/>
<input name="colorchg" type="hidden" value=""/>
<input name="com

In [3]:
a = [[2353, 2017, 3064], [4, 5, 6]]
print(a[0][0])
print(a[1])



























































































2353
[4, 5, 6]
